In [148]:
import numpy as np
import scipy
import pyscf
import pyscf.tools

import orbitalpartitioning

In [149]:
molecule = """
H 0.0 0.0 0.0
H 2.0 0.0 0.0
H 4.0 0.0 0.0
He 0.0 1.0 0.0
He 2.0 1.0 0.0
He 4.0 1.0 0.0
He 6.0 1.0 0.0
"""

basis = "def2-svp"
pymol = pyscf.gto.Mole(
        atom    =   molecule,
        symmetry=   True,
        spin    =   3, # number of unpaired electrons
        charge  =   0,
        basis   =   basis)


pymol.build()
print("symmetry: ",pymol.topgroup)
# mf = pyscf.scf.UHF(pymol).x2c()
mf = pyscf.scf.ROHF(pymol)
mf.verbose = 4
mf.conv_tol = 1e-8
mf.conv_tol_grad = 1e-5
mf.chkfile = "scf.fchk"
mf.init_guess = "sad"

mf.run(max_cycle=200)

print(" Hartree-Fock Energy: %12.8f" % mf.e_tot)
# mf.analyze()

# Get data
F = mf.get_fock()
C = mf.mo_coeff
S = mf.get_ovlp()
ndocc = mf.nelec[1]
nsing = mf.nelec[0] - ndocc
nvirt = mf.mol.nao - ndocc - nsing

# Just use alpha orbitals
Cdocc = mf.mo_coeff[:,0:ndocc]
Csing = mf.mo_coeff[:,ndocc:ndocc+nsing]
Cvirt = mf.mo_coeff[:,ndocc+nsing:ndocc+nsing+nvirt]

nbas = Cdocc.shape[0]

symmetry:  Cs


******** <class 'pyscf.scf.hf_symm.SymAdaptedROHF'> ********
method = SymAdaptedROHF-ROHF-RHF
initial guess = sad
damping factor = 0
level_shift factor = 0
DIIS = <class 'pyscf.scf.diis.CDIIS'>
diis_start_cycle = 1
diis_space = 8
SCF conv_tol = 1e-08
SCF conv_tol_grad = 1e-05
SCF max_cycles = 200
direct_scf = True
direct_scf_tol = 1e-13
chkfile to save SCF result = scf.fchk
max_memory 4000 MB (current use 0 MB)
num. doubly occ = 4  num. singly occ = 3
init E= -12.9159699126533
HOMO (A') = 0.073327776115038  LUMO (A') = 0.623823556262124
cycle= 1 E= -12.5806438958108  delta_E= 0.335  |g|= 0.201  |ddm|= 2.45
HOMO (A') = -0.0167359324955989  LUMO (A') = 0.612658246242397
cycle= 2 E= -12.6187473373525  delta_E= -0.0381  |g|= 0.0483  |ddm|= 0.672
HOMO (A') = 0.00733086580968545  LUMO (A') = 0.617347065628894
cycle= 3 E= -12.6212329972717  delta_E= -0.00249  |g|= 0.0146  |ddm|= 0.144
HOMO (A') = 0.0137545135518727  LUMO (A') = 0.616018802629854
cycle= 4 E= -12.6214563199353  

# Define Fragments by AOs

In [150]:
# Find AO's corresponding to atoms 
full = []
frag1 = []
frag2 = []
frag3 = []
for ao_idx,ao in enumerate(mf.mol.ao_labels(fmt=False)):
    if ao[0] in (0, 1, 2, 3):
        if ao[2] in ("1s",):
            frag1.append(ao_idx)
            full.append(ao_idx)
    elif ao[0] in (4, 5):
        if ao[2] in ("1s", ):
            frag2.append(ao_idx)
            full.append(ao_idx)
    elif ao[0] == 6:
        if ao[2] in ("1s", "2p"):
            frag3.append(ao_idx)
            full.append(ao_idx)


frags = [frag1, frag2, frag3]
print(frags)


[[0, 5, 10, 15], [20, 25], [30, 32, 33, 34]]


# Define Projectors
We can choose to project onto the non-orthogonal AOs, or onto the symmetrically orthogonalized AOs.

In [151]:
# Define projectors
X = np.eye(nbas) 
X = scipy.linalg.sqrtm(S)
Pfull = X[:,full]  # non-orthogonal
Pf = []
for f in frags:
    Pf.append(X[:,f])


# Project MOs onto all fragments
For each orbital block (Docc, Sing, Virt), project each subspace onto the full list of fragment AOs. This will determine our full active space.

In [152]:
(Oact, Sact, Vact), (Cenv, Cerr, _) = orbitalpartitioning.svd_subspace_partitioning((Cdocc, Csing, Cvirt), Pfull, S)
assert(Cerr.shape[1] == 0)
Cact = np.hstack((Oact,Vact))

 Partition   35 orbitals into a total of   10 orbitals
            Index   Sing. Val. Space       
                0   1.00003222            2*
                1   0.99999963            2*
                2   0.99938730            2*
                3   0.92063974            1*
                4   0.86547922            1*
                5   0.83131835            0*
                6   0.81677897            1*
                7   0.77288428            0*
                8   0.75149954            0*
                9   0.73304633            0*
               10   0.31774136            2
               11   0.30630899            2
               12   0.28726661            2
               13   0.15488648            2
               14   0.07264573            2
               15   0.05922181            2
               16   0.04157758            2


# Split active space into fragments

In [153]:
# Project active orbitals onto fragments
init_fspace = []
clusters = []
Cfrags = []
orb_index = 1
for fi,f in enumerate(frags):
    print()
    print(" Fragment: ", f)
    (Of, Sf, Vf), (_, _, _) = orbitalpartitioning.svd_subspace_partitioning((Oact, Sact, Vact), Pf[fi], S)
    Cfrags.append(np.hstack((Of, Sf, Vf)))
    ndocc_f = Of.shape[1]
    init_fspace.append((ndocc_f+Sf.shape[1], ndocc_f))
    nmof = Of.shape[1] + Sf.shape[1] + Vf.shape[1]
    clusters.append(list(range(orb_index, orb_index+nmof)))
    orb_index += nmof



# Orthogonalize Fragment orbitals
Cfrags = orbitalpartitioning.sym_ortho(Cfrags, S)

Cact = np.hstack(Cfrags)

# Write Molden files for visualization
pyscf.tools.molden.from_mo(mf.mol, "Pfull.molden", Pfull)
pyscf.tools.molden.from_mo(mf.mol, "Cact.molden", Cact)
pyscf.tools.molden.from_mo(mf.mol, "Cenv.molden", Cenv)
for i in range(len(frags)):
    pyscf.tools.molden.from_mo(mf.mol, "Cfrag%i.molden"%i, Cfrags[i])

print(" init_fspace = ", init_fspace)
print(" clusters    = ", clusters)



 Fragment:  [0, 5, 10, 15]
 Partition   35 orbitals into a total of    4 orbitals
            Index   Sing. Val. Space       
                0   0.87092211            1*
                1   0.75316009            0*
                2   0.61794346            1*
                3   0.53242844            1*
                4   0.45988260            0
                5   0.41034120            0
                6   0.30845594            2
                7   0.17574227            2
                8   0.12623863            2
                9   0.05548682            2
               10   0.00012128            0

 Fragment:  [20, 25]
 Partition   35 orbitals into a total of    2 orbitals
            Index   Sing. Val. Space       
                0   0.79085648            0*
                1   0.76583704            0*
                2   0.11375885            2
                3   0.11007515            2
                4   0.06916724            1
                5   0.05040802            

# Make Integrals

In [154]:
print(Cenv.shape)
print(Cact.shape)
d1_embed = 2 * Cenv @ Cenv.T

h0 = pyscf.gto.mole.energy_nuc(mf.mol)
h  = pyscf.scf.hf.get_hcore(mf.mol)
j, k = pyscf.scf.hf.get_jk(mf.mol, d1_embed, hermi=1)

print(h.shape)

(35, 0)
(35, 10)
(35, 35)


In [155]:
h0 += np.trace(d1_embed @ ( h + .5*j - .25*k))

h = Cact.T @ h @ Cact;
j = Cact.T @ j @ Cact;
k = Cact.T @ k @ Cact;

In [156]:
nact = h.shape[0]

h2 = pyscf.ao2mo.kernel(pymol, Cact, aosym="s4", compact=False)
h2.shape = (nact, nact, nact, nact)

In [157]:
# The use of d1_embed only really makes sense if it has zero electrons in the
# active space. Let's warn the user if that's not true

S = pymol.intor("int1e_ovlp_sph")
n_act = np.trace(S @ d1_embed @ S @ Cact @ Cact.T)
if abs(n_act) > 1e-8 == False:
    print(n_act)
    error(" I found embedded electrons in the active space?!")

h1 = h + j - .5*k;


In [158]:
np.save("ints_h0", h0)
np.save("ints_h1", h1)
np.save("ints_h2", h2)
np.save("mo_coeffs", Cact)
np.save("overlap_mat", S)

Pa = Cact.T @ S @ mf.make_rdm1()[0] @ S @ Cact
Pb = Cact.T @ S @ mf.make_rdm1()[1] @ S @ Cact
np.save("Pa", Pa)
np.save("Pb", Pb)

In [159]:
import numpy as np
Ccmf = np.load("Ccmf.npy")
pyscf.tools.molden.from_mo(mf.mol, "Ccmf.molden", Ccmf)

# Recompute ROHF

In [160]:
e, U = np.linalg.eigh(Cact.T @ F @ Cact)
Ccanon = Cact@U 
print(e)
pyscf.tools.molden.from_mo(mf.mol, "Cact_canon.molden", Ccanon)
na = 7
nb = 4
Pa = Ccanon[:,0:na] @ Ccanon[:,0:na].T
Pb = Ccanon[:,0:nb] @ Ccanon[:,0:nb].T
# Pa += Cenv@Cenv.T
# Pb += Cenv@Cenv.T
Pmf = mf.make_rdm1()
print(np.trace(Pmf[0] @ S))
print(np.trace(Pa@S))
print(np.trace(Pmf[1] @ S))
print(np.trace(Pb@S))

print("PaPa : ", np.trace(Pa@S@Pmf[0]@S))
print("PbPb : ", np.trace(Pb@S@Pmf[1]@S))

H  = pyscf.scf.hf.get_hcore(mf.mol)
J, K = pyscf.scf.hf.get_jk(mf.mol, Cenv@Cenv.T, hermi=1)
h2 = pyscf.ao2mo.kernel(pymol, Cact, aosym="s4", compact=False)
h2.shape = (nact, nact, nact, nact)

ea = np.trace((H+J-K)@Pa )
eb = np.trace((H+J-K)@Pb + J@Pa)
# Ehf = -15.7323701829171
print(mf.energy_nuc())
print((ea + eb))
print((ea + eb)+mf.energy_nuc())

[-1.07809959 -1.04553746 -1.01744175 -0.91984778 -0.13864773 -0.07598431
  0.01332604  1.96938987  1.9780368   1.97893821]
6.999999999999999
7.0000000000000036
4.0
4.0000000000000036
PaPa :  7.0000000000000036
PbPb :  3.99999999999996
11.73334805412517
-39.476518651802934
-27.743170597677764
